# Clib Raster 

**Purpose** : 
> Merge multiple title and clip to our AOI.

<hr/> 

## 1-import required libraries

In [69]:
# Import necessary packages
import os, glob, functools, fnmatch, requests,  io, shutil, tarfile, json
from pathlib import Path
from zipfile import ZipFile
from itertools import product
from configparser import ConfigParser
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import rasterio 
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon, mapping
from rasterio.mask import mask
from folium import Map, GeoJson, Figure
import cv2
from configparser import ConfigParser
from shapely.geometry import box
%matplotlib inline
sns.set(font_scale=1.5, style="white")

<hr/> 

## 2- Plot the AOI

In [70]:
#Set the path to aoi data
aoi_dir = r"/media/duplex/COMMUN/ML2023/AI LULC/region/region/cmr_aoi_01"
aoi_file = "littoral_outline.geojson"
# Load the site boundaries into a geopandas dataframe 
aoi_path = os.path.join(aoi_dir  , aoi_file)
gdf = gpd.read_file(aoi_path )
#Uncomment this line if the crs is different
#gdf  = gdf.to_crs("EPSG:4326")
# Get AOI bounds
bbox_aoi = gdf.geometry.total_bounds
# Get AOI centroid for plotting with folium
centroid_aoi = [box(*bbox_aoi).centroid.x, box(*bbox_aoi).centroid.y]

In [71]:
#Get the bounding extend to 200 m
def get_bounds_of_AoI(obj_aoi, offset):
    
    aoi = gpd.read_file(obj_aoi)
    
    bounds = aoi.total_bounds
    
    #offset = 1/60  #200m in degree

    # Extend the bounding box by 200 m
    minx, miny = bounds[0]-offset, bounds[1]-offset
    maxx, maxy = bounds[2]+offset, bounds[3]+offset

    bbox = box(minx, miny, maxx, maxy)
    
    print(bbox)

    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs="EPSG:4326")

    
    return geo 

In [72]:
bbx = get_bounds_of_AoI(aoi_path, 1/60)

POLYGON ((11.094414006666668 3.243667343333333, 11.094414006666668 5.351516026666666, 9.319134458233293 5.351516026666666, 9.319134458233293 3.243667343333333, 11.094414006666668 3.243667343333333))


In [73]:
m = Map(tiles="OpenStreetMap",
        control_scale=True,
        location = [centroid_aoi[1], centroid_aoi[0]],
        zoom_start = 10,
        max_zoom = 20,
        min_zoom =6,
        width = '100%',
        height = '100%',
        zoom_control=False )
GeoJson(gdf).add_to(m)
GeoJson(bbx).add_to(m)
Figure(width=800, height=300).add_child(m)

<hr/> 

## 4-Clib the mosaic to our AOI

In [74]:
def get_bounds_of_AoI_coordinate(obj_aoi, src_crs):
    
    aoi = gpd.read_file(obj_aoi)
    
    bounds = aoi.total_bounds
    
    offset = 1/60  #200m in degree

    # WGS84 coordinates
    minx, miny = bounds[0]-offset, bounds[1]-offset
    maxx, maxy = bounds[2]+offset, bounds[3]+offset

    bbox = box(minx, miny, maxx, maxy)
    
    print(bbox)

    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs="EPSG:4326")

    #picking up the coordinate system of the image:
    #crs=src.crs.to_epsg()
    geo = geo.to_crs(crs=src_crs) #src.crs.to_epsg())

    coords = getFeatures(geo)
    
    return coords

    print(coords)

In [75]:
def clip_raster_with_bounds(in_tif, out_tif, coords):

    #load the mosaided file
    data = rasterio.open(in_tif)

    out_img, out_transform = rasterio.mask.mask(dataset=data, shapes=coords, crop=True)

    # Copy the metadata
    out_meta = data.meta.copy()

    # Parse EPSG code
    epsg_code = int(data.crs['init'][5:])

    out_meta.update({"driver": "GTiff",
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform,
            "crs": epsg_code} #pycrs.parse.from_epsg_code(epsg_code).to_proj4()}
            )

    with rasterio.open(out_tif, "w", **out_meta) as dest:
        dest.write(out_img)

    print(out_tif)

In [76]:
def get_src_crs(prototype_tif):
    
    with rasterio.open(prototype_tif, 'r') as test:
        src_crs = test.crs.to_epsg()
        
    return src_crs


In [77]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [78]:
#The folder to save RGBNIR mosaic on a temporary folder 
temp_folder = r"/media/duplex/COMMUN/ML2023/AI LULC"
#temp_file = "ESA_WC_2020.tif"
temp_file = "ESA_WC_2021.tif"
out_mosaic = os.path.join(temp_folder,temp_file )
# Write the mosaic to raster file
esa_wc_2020 = rasterio.open(out_mosaic)

In [79]:
esa_wc_2020.shape

(36000, 36000)

In [80]:
# Set directory for downloading the  tiles to
output_dir = r"/media/duplex/COMMUN/ML2023/AI LULC"
#output_file = "ESA_WC_2020_clip.tif"
output_file = "ESA_WC_2021_clip.tif"
#mosaic_clip_path  = os.path.join(output_dir , output_file) 
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
#Set output path 
image_file_out = os.path.join(output_dir,output_file)
#get the source crs
src_crs = get_src_crs(out_mosaic)
#Get the bounding box + 200 m coordinate
coords = get_bounds_of_AoI_coordinate(aoi_path,src_crs)
#Clib the raster to our AOI
clip_raster_with_bounds(out_mosaic,image_file_out, coords)

POLYGON ((11.094414006666668 3.243667343333333, 11.094414006666668 5.351516026666666, 9.319134458233293 5.351516026666666, 9.319134458233293 3.243667343333333, 11.094414006666668 3.243667343333333))
/media/duplex/COMMUN/ML2023/AI LULC/ESA_WC_2021_clip.tif


In [81]:
#The folder to save RGBNIR mosaic on a temporary folder 
temp_folder = r"/media/duplex/COMMUN/ML2023/AI LULC"
temp_file = "ESA_WC_2020_clip.tif"
out_mosaic = os.path.join(temp_folder,temp_file )
# Write the mosaic to raster file
#esa_wc_2020_clip = rasterio.open(out_mosaic)
esa_wc_2021_clip = rasterio.open(out_mosaic)

In [82]:
#esa_wc_2020_clip.shape
esa_wc_2021_clip.shape

(25295, 21304)